In [1]:
import numpy as np
import cv2

In [2]:
def erod(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross' or type_of_mask == 'vertline' or type_of_mask == 'horizline'
    result = image.copy()
    if type_of_mask == 'square':
        result = erod_square(result, size)
    elif type_of_mask == 'cross':
        result = erod_cross(result, size)
    elif type_of_mask == 'vertline':
        result = erod_vert_line(result, size)
    elif type_of_mask == 'horizline':
        result = erod_horiz_line(result, size)
    return result

In [3]:
def process_operation(image, i, j, mask, operation):
    offset = int(mask.shape[0] / 2)
    mask_i = 0
    mask_j = 0
    right_border = offset
    if mask.shape[0] % 2 != 0:
        right_border += 1
    if operation == 'erod':
        for k in range(i - offset, i + right_border):
            mask_j = 0
            for l in range(j - offset, j + right_border):
                if mask[mask_i, mask_j] == 1 and image[k, l] != 0:
                    return 255
                mask_j += 1
            mask_i += 1
    elif operation == 'dilat':
        for k in range(i - offset, i + right_border):
            mask_j = 0
            for l in range(j - offset, j + right_border):
                if mask[mask_i, mask_j] == 1 and image[k, l] == 0:
                    return 0
                mask_j += 1
            mask_i += 1
        return 255
    return 0
    

In [57]:
def erod_square(image, size):
    result = image.copy()
    status = True
    for i in range(size - 1, result.shape[0] - size):
        for j in range(size - 1, result.shape[1] - size):
            result[i, j] = process_operation(image, i, j, np.ones((size, size)), 'erod')
    return result

In [58]:
def erod_cross(image, size):
    result = image.copy()
    status = True
    mask = create_cross(size)
    for i in range(size - 1, result.shape[0] - size):
        for j in range(size - 1, result.shape[1] - size):
            result[i, j] = process_operation(image, i, j, mask, 'erod')
    return result

In [6]:
def create_cross(size):
    mask = np.zeros((size, size))
    center_point = int(size / 2)
    for i in range(size):
        mask[center_point, i] = 1
    for i in range(size):
        mask[i, center_point] = 1
    return mask

In [7]:
def dilat(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross'
    
    result = image.copy()
    if type_of_mask == 'square':
        result = dilat_square(result, size)
    elif type_of_mask == 'cross':
        result = dilat_cross(result, size)
    return result

In [59]:
def dilat_square(image, size):
    result = image.copy()
    status = False
    for i in range(size - 1, result.shape[0] - size):
        for j in range(size - 1, result.shape[1] - size):
            result[i, j] = process_operation(image, i, j, np.ones((size, size)), 'dilat')
    return result

In [60]:
def dilat_cross(image, size):
    result = image.copy()
    status = False
    mask = create_cross(size)
    for i in range(size - 1, result.shape[0] - size):
        for j in range(size - 1, result.shape[1] - size):
            result[i, j] = process_operation(image, i, j, mask, 'dilat')
    return result

In [10]:
def opening(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross'
    
    result = image.copy()
    result = erod(result, size, type_of_mask)
    result = dilat(result, size, type_of_mask)
    return result

In [11]:
def closing(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross'
    
    result = image.copy()
    result = dilat(result, size, type_of_mask)
    result = erod(result, size, type_of_mask)
    return result

In [12]:
def filter(image):
    result = image.copy()
    for i in range(3, image.shape[0] - 3):
        for j in range(3, image.shape[1] - 3):
            if image[i, j] == 0:
                if image[i, j + 1] == 255 and image[i, j - 1] == 255 and image[i + 1, j] == 255 and image[i - 1, j] == 255:
                    image[i, j] = 255
            elif image[i, j] == 255:
                if image[i, j + 1] == 0 and image[i, j - 1] == 0 and image[i + 1, j] == 0 and image[i - 1, j] == 0:
                    image[i, j] = 0
    return result

In [13]:
def make_noise(image, prob):
    result = image.copy()
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            p = np.random.randint(0, 101)
            if p < prob :
                p_color = np.random.randint(0, 101)
                if p_color < 50:
                    result[i, j] = 0
                else:
                    result[i, j] = 255
    return result

In [14]:
def circle(image, centre, radius):
    circle = [centre, radius]
    for i in range(circle[0][0] - circle[1], circle[0][0] + circle[1]):
        for j in range(circle[0][1] - circle[1], circle[0][1] + circle[1]):
            if (i - circle[0][0]) ** 2 + (j - circle[0][1]) ** 2 < circle[1] ** 2:
                image[i, j] = 0

In [15]:
def triangle(image, A, B, C):
    triangle = [A, B, C]
    x_a = triangle[0][1]
    y_a = triangle[0][0]
    x_b = triangle[1][1]
    y_b = triangle[1][0]
    x_c = triangle[2][1]
    y_c = triangle[2][0]

    left = triangle[0][0]
    right = triangle[0][0]
    above = triangle[0][1]
    below = triangle[0][1]
    for i in triangle:
        if left > i[0]:
            left = i[0]
        if right < i[0]:
            right = i[0]
        if above > i[1]:
            above = i[1]
        if below < i[1]:
            below = i[1]
    for i in range(above, below):
        for j in range(left, right):
            coord_1 = (x_a - i) * (y_b - y_a) - (x_b - x_a) * (y_a - j)
            coord_2 = (x_b - i) * (y_c - y_b) - (x_c - x_b) * (y_b - j)
            coord_3 = (x_c - i) * (y_a - y_c) - (x_a - x_c) * (y_c - j)
            if coord_1 >= 0 and coord_2 >= 0 and coord_3 >= 0:
                image[i, j] = 0

In [16]:
def square(image, A, B):
    square = [A, B]
    for i in range(square[0][0], square[1][0]):
        for j in range(square[0][1], square[1][1]):
            image[i, j] = 0

In [17]:
def noise_reduce(image, filter_image, noised_image):
    assert image.shape == filter_image.shape and image.shape == noised_image.shape
    count_noised = 0
    count_filtered = 0
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if image[i, j] - noised_image[i, j] != 0:
                count_noised += 1
            if image[i, j] - filter_image[i, j] != 0:
                count_filtered += 1
    return count_noised / count_filtered

# 1. Фильтрация бинарных изображений

## 1.1. Создать или выбрать бинарное изображение с любым простым рисунком, состоящим из примитивных фигур (круги, квадраты, полигоны).

In [47]:
image = np.zeros((500, 500))
image.fill(255)

In [48]:
square(image, (10, 10), (50, 50))
triangle(image, (100, 100), (100, 200), (200, 100))
circle(image, (400, 400), 40)
circle(image, (200, 250), 1)

In [49]:
path = '/home/sergei/'
cv2.imwrite(path + 'image.png', image)

True

## 1.2. На полученное изображение наложить шум (1%, 2%, 5%, 10%, 20%, 50%).

In [51]:
noised = []
for i in [1, 2, 5, 10, 20, 50]:
    noise = make_noise(image, i)
    noised.append(noise)
    cv2.imwrite(path + 'noise/image_noise_' + str(i) + '%' + '.png', noise)

## 1.3. Выполнить операции эрозии и дилатации для различных структурных элементов (крест, квадрат). Визуально оценить результаты эрозии и дилатации.

In [61]:
for i in range(3, 6):
    erod_image = erod(image, i, type_of_mask = 'square')
    cv2.imwrite(path + 'erod/erod_image_square_size' + str(i) + '.png', erod_image)

In [62]:
for i in range(3, 6):
    erod_image = erod(image, i, type_of_mask = 'cross')
    cv2.imwrite(path + 'erod/erod_image_cross_size'+ str(i) + '.png', erod_image)

In [63]:
for i in range(3, 6):
    dilat_image = dilat(image, i, type_of_mask = 'square')
    cv2.imwrite(path + 'dilat/dilat_image_square_size' + str(i) + '.png', dilat_image)

In [64]:
for i in range(3, 6):
    dilat_image= dilat(image, i, type_of_mask = 'cross')
    cv2.imwrite(path + 'dilat/dilat_image_cross_size' + str(i) + '.png', dilat_image)

## 1.4. Провести фильтрацию шумов на изображении при помощи морфологических операций вскрытия и закрытия с различными размерами (от 3 до 5) и формами структурного элемента (крест, квадрат), а также алгоритма фильтрации шумов, которому соответствует логическая таблица истинности, представленная в табл. 1 со крестообразным структурным элементом размером 3.

In [83]:
noise_index = [1, 2, 5, 10, 20, 50]

In [84]:
opened_image_square = np.zeros((3, len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(3, 6):
    for j in range(len(noised)):
        open_image = opening(noised[j], i, type_of_mask = 'square')
        opened_image_square[i - 3, j] = open_image
        cv2.imwrite(path + 'open/open_image_square_noise' + str(noise_index[j]) + '%_size' + str(i) + '.png', open_image)

In [85]:
closed_image_square = np.zeros((3, len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(3, 6):
    for j in range(len(noised)):
        close_image = closing(noised[j], i, type_of_mask = 'square')
        closed_image_square[i - 3, j] = close_image
        cv2.imwrite(path + 'close/close_image_square' + str(noise_index[j]) + '%_size' + str(i) + '.png', close_image)

In [86]:
opened_image_cross = np.zeros((3, len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(3, 6):
    for j in range(len(noised)):
        open_image = opening(noised[j], i, type_of_mask = 'cross')
        opened_image_cross[i - 3, j] = open_image
        cv2.imwrite(path + 'open/open_image_cross' + str(noise_index[j]) + '%_size' + str(i) + '.png', open_image)

In [87]:
closed_image_cross = np.zeros((3, len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(3, 6):
    for j in range(len(noised)):
        close_image = closing(noised[j], i, type_of_mask = 'cross')
        closed_image_cross[i - 3, j] = close_image
        cv2.imwrite(path + 'close/close_image_cross' + str(noise_index[j]) + '%_size' + str(i) + '.png', close_image)

In [88]:
filtered_image = np.zeros((3, len(noised), noised[0].shape[0], noised[0].shape[1]))
for i in range(len(noised)):
    filter_image = filter(noised[i])
    filtered_image[i - 3, j] = filter_image
    cv2.imwrite(path + 'filter_image_' + str(i) + '.png', filter_image)

## 1.5. Рассчитать коэффициент снижения шума для различных методов фильтрации, для различной интенсивности шума и различных размеров и форм структурного элемента (окна).

In [70]:
for i in range(opened_image_square.shape[0]):
    for j in range(opened_image_square.shape[1]):
        print('Size {} Noised {} reduce {}'.format(i + 3, j, noise_reduce(image, opened_image_square[i, j], noised[j])))

Size 3 Noised 0 reduce 0.5362318840579711
Size 3 Noised 1 reduce 0.6039215686274509
Size 3 Noised 2 reduce 0.8846153846153846
Size 3 Noised 3 reduce 1.6065146579804561
Size 3 Noised 4 reduce 2.367899249732047
Size 3 Noised 5 reduce 3.972843899241502
Size 4 Noised 0 reduce 0.07621009268795056
Size 4 Noised 1 reduce 0.13095238095238096
Size 4 Noised 2 reduce 0.3761985335589397
Size 4 Noised 3 reduce 0.8048302872062664
Size 4 Noised 4 reduce 1.27665414620052
Size 4 Noised 5 reduce 2.882788611809472
Size 5 Noised 0 reduce 0.19680851063829788
Size 5 Noised 1 reduce 0.21782178217821782
Size 5 Noised 2 reduce 0.3501312335958005
Size 5 Noised 3 reduce 0.5250159676389184
Size 5 Noised 4 reduce 0.8511847428241187
Size 5 Noised 5 reduce 2.5038951841359776


In [71]:
for i in range(closed_image_square.shape[0]):
    for j in range(closed_image_square.shape[1]):
        print('Size {} Noised {} reduce {}'.format(i + 3, j, noise_reduce(image, closed_image_square[i, j], noised[j])))

Size 3 Noised 0 reduce 0.06156405990016639
Size 3 Noised 1 reduce 0.06182256122039342
Size 3 Noised 2 reduce 0.0963316002310803
Size 3 Noised 3 reduce 0.14612467409338706
Size 3 Noised 4 reduce 0.19475482093663912
Size 3 Noised 5 reduce 0.2716585347112836
Size 4 Noised 0 reduce 0.036742800397219465
Size 4 Noised 1 reduce 0.04488487321480618
Size 4 Noised 2 reduce 0.07139798758295868
Size 4 Noised 3 reduce 0.09402524116368628
Size 4 Noised 4 reduce 0.11031495374936023
Size 4 Noised 5 reduce 0.202512673152011
Size 5 Noised 0 reduce 0.05653170359052712
Size 5 Noised 1 reduce 0.04975767366720517
Size 5 Noised 2 reduce 0.05421882620712079
Size 5 Noised 3 reduce 0.05791723425243083
Size 5 Noised 4 reduce 0.06824254405609527
Size 5 Noised 5 reduce 0.1889656462539585


In [72]:
for i in range(opened_image_cross.shape[0]):
    for j in range(opened_image_cross.shape[1]):
        print('Size {} Noised {} reduce {}'.format(i + 3, j, noise_reduce(image, opened_image_cross[i, j], noised[j])))

Size 3 Noised 0 reduce 0.6434782608695652
Size 3 Noised 1 reduce 0.7979274611398963
Size 3 Noised 2 reduce 1.2306273062730628
Size 3 Noised 3 reduce 2.3176691729323307
Size 3 Noised 4 reduce 3.8371689101172386
Size 3 Noised 5 reduce 5.329899497487437
Size 4 Noised 0 reduce 0.10249307479224377
Size 4 Noised 1 reduce 0.19154228855721392
Size 4 Noised 2 reduce 0.6730575176589304
Size 4 Noised 3 reduce 1.8402985074626865
Size 4 Noised 4 reduce 3.3549734244495064
Size 4 Noised 5 reduce 4.781471880987265
Size 5 Noised 0 reduce 0.3627450980392157
Size 5 Noised 1 reduce 0.4489795918367347
Size 5 Noised 2 reduce 0.7728852838933952
Size 5 Noised 3 reduce 1.538365564566438
Size 5 Noised 4 reduce 2.4899971823048745
Size 5 Noised 5 reduce 3.9026768466562416


In [73]:
for i in range(closed_image_cross.shape[0]):
    for j in range(closed_image_cross.shape[1]):
        print('Size {} Noised {} reduce {}'.format(i + 3, j, noise_reduce(image, closed_image_cross[i, j], noised[j])))

Size 3 Noised 0 reduce 0.0631938514090521
Size 3 Noised 1 reduce 0.06617963042544048
Size 3 Noised 2 reduce 0.11341608569971093
Size 3 Noised 3 reduce 0.20166830225711482
Size 3 Noised 4 reduce 0.3202275692129294
Size 3 Noised 5 reduce 0.4494041629150998
Size 4 Noised 0 reduce 0.10466760961810467
Size 4 Noised 1 reduce 0.19948186528497408
Size 4 Noised 2 reduce 0.4951744617668894
Size 4 Noised 3 reduce 0.6932808546527973
Size 4 Noised 4 reduce 0.5554717455528317
Size 4 Noised 5 reduce 0.3966863329936139
Size 5 Noised 0 reduce 0.06239460370994941
Size 5 Noised 1 reduce 0.06536502546689305
Size 5 Noised 2 reduce 0.11299339318990344
Size 5 Noised 3 reduce 0.19969228277593326
Size 5 Noised 4 reduce 0.28902698282910877
Size 5 Noised 5 reduce 0.3161966372525638


In [74]:
for i in range(filtered_image.shape[0]):
    for j in range(filtered_image.shape[1]):
        print('Size {} Noised {} reduce {}'.format(i + 3, j, noise_reduce(image, filtered_image[i, j], noised[j])))

Size 3 Noised 0 reduce 0.000310615059793399
Size 3 Noised 1 reduce 0.0006464151244349115
Size 3 Noised 2 reduce 0.00279973303894861
Size 3 Noised 3 reduce 0.01035103699257462
Size 3 Noised 4 reduce 0.03709331464046307
Size 3 Noised 5 reduce 3.482679362994582
Size 4 Noised 0 reduce 0.000310615059793399
Size 4 Noised 1 reduce 0.0006464151244349115
Size 4 Noised 2 reduce 0.00279973303894861
Size 4 Noised 3 reduce 0.01035103699257462
Size 4 Noised 4 reduce 0.03709331464046307
Size 4 Noised 5 reduce 1.7238633131526553
Size 5 Noised 0 reduce 0.000310615059793399
Size 5 Noised 1 reduce 0.0006464151244349115
Size 5 Noised 2 reduce 0.00279973303894861
Size 5 Noised 3 reduce 0.01035103699257462
Size 5 Noised 4 reduce 0.03709331464046307
Size 5 Noised 5 reduce 0.693076747149345


# 2. Выделение контуров на бинарных изображениях

## 2.1. Создать или выбрать бинарное изображение, содержащее простой рисунок, состоящий из примитивных фигур (полосы, круги, квадраты).

In [75]:
image = np.zeros((500, 500))
image.fill(255)

In [76]:
square(image, (10, 10), (50, 50))
triangle(image, (100, 100), (100, 200), (200, 100))
circle(image, (400, 400), 40)
circle(image, (200, 250), 1)

In [77]:
cv2.imwrite(path + 'image.png', image)

True

## 2.2. С помощью морфологических операций выделить контур объекта. Выяснить, когда контур получается внешним, внутренним, четырёхсвязным, восьмисвязным.

In [78]:
def border(image, mask_type):
    assert mask_type == 'square' or mask_type == 'cross' or mask_type == 'vertline' or mask_type == 'horizline'
    
    result = image.copy()
    erod_image = erod(image, 3, mask_type)
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            if image[i, j] - erod_image[i, j] != 0:
                result[i, j] = 0
            else:
                result[i, j] = 255
    return result

In [79]:
b = border(image, 'square')
cv2.imwrite(path + 'b.png', b)

True

In [80]:
def erod_vert_line(image, size):
    result = image.copy()
    status = True
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            for k in range(1, int((size + 1) / 2)):
                status = (status and image[i, j - k] == 0 and image[i, j + k] == 0)
                if not status:
                    break
            if not status:
                result[i, j] = 255
            status = True
    return result

In [81]:
b = border(image, 'vertline')
cv2.imwrite(path + 'b.png', b)

True

In [82]:
def erod_horiz_line(image, size):
    result = image.copy()
    status = True
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            for k in range(1, int((size + 1) / 2)):
                status = (status and image[i - 1, j] == 0 and image[i + 1, j] == 0)
                if not status:
                    break
            if not status:
                result[i, j] = 255
            status = True
    return result